# 第6章 入力処理: チェーン式プロンプト Chaining Prompts

 - [一、環境設定](#一、環境設定)
 - [二、 複数のプロンプトを含む複雑なタスクの実装](#二、-複数のプロンプトを含む複雑なタスクの実装)
     - [2.1 関連製品とカテゴリー名の抽出](#2.1-関連製品とカテゴリー名の抽出)
     - [2.2 抽出された製品とカテゴリーの詳細情報の取得](#2.2-抽出された製品とカテゴリーの詳細情報の取得)
     - [2.3 Python文字列をPython辞書リストとして読み込む](#2.3-Python文字列をPython辞書リストとして読み込む)
     - [2.4 詳細な製品情報に基づくユーザークエリへの回答生成](#2.4-詳細な製品情報に基づくユーザークエリへの回答生成)

本章では、複雑なタスクを一連の単純なサブタスクに分割することで、複数のプロンプトをチェーンする方法を学びます。

なぜタスクを複数のプロンプトに分割するのか、前回のビデオで学んだように思考の連鎖推論を使用して一度に完了させるのではなく、と疑問に思うかもしれません。言語モデル、特にGPT-4のような高度なモデルは、複雑な指示に従うのが非常に得意であることを証明しました。

それでは、なぜこのようにするのか、思考の連鎖推論とチェーン式プロンプトを比較するために2つの比喩を使って説明しましょう。

タスクを複数のプロンプトに分割する最初の比喩は、複雑な料理を一度に調理することと段階的に調理することの違いです。長くて複雑なプロンプトを使用することは、複雑な料理を一度に調理するようなもので、複数の材料、調理技術、時間を同時に管理しなければなりません。これは非常に挑戦的で、各部分を追跡し、各構成要素が適切に調理されることを確認するのが困難です。一方、チェーン式プロンプトは食事を段階的に調理するようなもので、一つの構成要素に集中し、各部分が正しく調理されてから次に進みます。このアプローチはタスクの複雑さを分解し、より管理しやすくし、エラーの可能性を減らします。ただし、非常に簡単なレシピの場合、このアプローチは不要で過度に複雑になる可能性があります。

もう少し良い比喩は、すべてのタスクを一度に完了することと段階的に完了することの違いです。長いコードの一覧を読むことと単純なモジュール化されたプログラムを使用することの違いのように、複雑な依存関係によってコードが混乱し、デバッグが困難になることがあります。この比喩は、複雑な単一ステップのタスクを言語モデルに提出する場合にも同様に適用されます。任意の時点でシステムの状態を維持し、現在の状態に基づいて異なるアクションを実行できるワークフローがある場合、チェーン式プロンプトは強力な戦略となります。

## 一、環境設定

前章と同様に、まずOpenAI APIを使用する環境を設定する必要があります

In [ ]:
import openai
# サードパーティライブラリのインポート

openai.api_key  = "sk-..."
# API_KEYの設定、ご自身のAPI_KEYに置き換えてください

# 以下は環境変数を使用した設定方法の例です。より安全です。参考までに、以降は触れません。
# import openai
# import os
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    '''
    OpenAI GPT3.5にアクセスする関数をカプセル化

    パラメータ: 
    messages: これはメッセージのリストで、各メッセージはrole（役割）とcontent（内容）を含む辞書です。役割は'system'、'user'、または'assistant'になります。内容は役割のメッセージです。
    model: 呼び出すモデル、デフォルトはgpt-3.5-turbo（ChatGPT）、内部テスト資格を持つユーザーはgpt-4を選択できます
    temperature: これはモデル出力のランダム性を決定します。デフォルトは0で、出力が非常に確定的になります。温度を上げると出力がよりランダムになります。
    max_tokens: これはモデル出力の最大トークン数を決定します。
    '''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # モデル出力のランダム性を決定
        max_tokens=max_tokens, # モデル出力の最大トークン数を決定
    )
    return response.choices[0].message["content"]

## 二、 複数のプロンプトを含む複雑なタスクの実装

### 2.1 関連製品とカテゴリー名の抽出

顧客のクエリを分類した後、クエリのカテゴリー（アカウントの問題か製品の問題か）を取得します。その後、異なるカテゴリーに基づいて異なるアクションを実行できます。

各サブタスクには対応するタスクを実行するために必要な指示のみが含まれており、これによりシステムの管理が容易になり、モデルがタスクを実行するために必要なすべての情報を持つことを確保し、エラーの可能性を減らします。この方法はコストも削減できます。より長いプロンプトとより多くのトークンは、より高い実行コストにつながり、場合によってはすべてのステップを概説する必要がないこともあります。

このアプローチのもう一つの利点は、どのステップが失敗しやすいか、または特定のステップで人間の介入が必要かをテストしやすいことです。

これらのモデルの構築と対話を深めていくにつれて、この戦略をいつ使用するかの直感を徐々に培っていきます。さらに、追加の利点として、必要に応じてモデルが外部ツールを使用できることです。例えば、製品カタログで何かを検索したり、APIを呼び出したり、知識ベースを検索したりすることができ、これは単一のプロンプトでは実現できません。

In [3]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Output a Python list of objects, where each object has \
the following format:
    'category': <one of Computers and Laptops, \
    Smartphones and Accessories, \
    Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, 
    Audio Equipment, Cameras and Camcorders>,
OR
    'products': <a list of products that must \
    be found in the allowed products below>

Where the categories and products must be found in \
the customer service query.
If a product is mentioned, it must be associated with \
the correct category in the allowed products list below.
If no products or categories are found, output an \
empty list.

Allowed products: 

Computers and Laptops category:
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

Smartphones and Accessories category:
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

Televisions and Home Theater Systems category:
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

Gaming Consoles and Accessories category:
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

Audio Equipment category:
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

Cameras and Camcorders category:
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

Only output the list of objects, with nothing else.
"""
user_message_1 = f"""
 tell me about the smartx pro phone and \
 the fotosnap camera, the dslr one. \
 Also tell me about your tvs """
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 
category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

[
    {'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']},
    {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']},
    {'category': 'Televisions and Home Theater Systems'}
]


In [ ]:
delimiter = "####"
system_message = f"""
サービスクエリが提供されます。
サービスクエリは{delimiter}文字で区切られます。

以下の形式で各オブジェクトを持つPythonオブジェクトリストのみを出力してください：
    'category': <コンピューターとノートパソコン、スマートフォンとアクセサリー、テレビとホームシアターシステム、ゲーム機とアクセサリー、オーディオ機器、カメラとビデオカメラのいずれか>,
または
    'products': <以下の許可された製品リストで見つかる必要がある製品のリスト>

カテゴリーと製品は顧客サービスクエリで見つかる必要があります。
製品が言及されている場合は、許可された製品リストの正しいカテゴリーに関連付ける必要があります。
製品またはカテゴリーが見つからない場合は、空のリストを出力してください。

許可された製品：

コンピューターとノートパソコンカテゴリー：
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

スマートフォンとアクセサリーカテゴリー：
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

テレビとホームシアターシステムカテゴリー：
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

ゲーム機とアクセサリーカテゴリー：
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

オーディオ機器カテゴリー：
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

カメラとビデオカメラカテゴリー：
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

Pythonオブジェクトリストのみを出力し、他の文字情報は含めないでください。
"""
user_message_1 = f"""
 SmartX ProPhoneスマートフォンとFotoSnapカメラ、一眼レフカメラについて教えてください。
 また、テレビ製品についての情報も教えてください。 """
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 
category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

ご覧のとおり、出力はオブジェクトのリストで、各オブジェクトにはカテゴリーといくつかの製品があります。「SmartX ProPhone」と「Fotosnap DSLR Camera」など。

最後のオブジェクトでは、具体的なテレビについて言及されていないため、カテゴリーのみがあります。

この構造化された応答出力の利点は、Pythonのリストに簡単に読み込むことができることです。

別の例を試してみましょう。

In [ ]:
user_message_2 = f"""
my router isn't working"""
messages =  [  
{'role':'system',
 'content': system_message},    
{'role':'user',
 'content': f"{delimiter}{user_message_2}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

[]


In [ ]:
user_message_2 = f"""私のルーターが壊れています"""
messages =  [  
{'role':'system',
 'content': system_message},    
{'role':'user',
 'content': f"{delimiter}{user_message_2}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

リストを注意深く見ると、実際にはルーターに関する情報が含まれていないことがわかります。

今、出力を完成させるために適切にフォーマットする必要があります。

ご覧のとおり、この場合、出力は空のリストです。

### 2.2 抽出された製品とカテゴリーの詳細情報の取得

大量の製品情報を例として提供し、モデルに製品と対応する詳細情報を抽出するよう要求します

In [5]:
# product information
products = {
    "TechPro Ultrabook": {
        "name": "TechPro Ultrabook",
        "category": "Computers and Laptops",
        "brand": "TechPro",
        "model_number": "TP-UB100",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["13.3-inch display", "8GB RAM", "256GB SSD", "Intel Core i5 processor"],
        "description": "A sleek and lightweight ultrabook for everyday use.",
        "price": 799.99
    },
    "BlueWave Gaming Laptop": {
        "name": "BlueWave Gaming Laptop",
        "category": "Computers and Laptops",
        "brand": "BlueWave",
        "model_number": "BW-GL200",
        "warranty": "2 years",
        "rating": 4.7,
        "features": ["15.6-inch display", "16GB RAM", "512GB SSD", "NVIDIA GeForce RTX 3060"],
        "description": "A high-performance gaming laptop for an immersive experience.",
        "price": 1199.99
    },
    "PowerLite Convertible": {
        "name": "PowerLite Convertible",
        "category": "Computers and Laptops",
        "brand": "PowerLite",
        "model_number": "PL-CV300",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["14-inch touchscreen", "8GB RAM", "256GB SSD", "360-degree hinge"],
        "description": "A versatile convertible laptop with a responsive touchscreen.",
        "price": 699.99
    },
    "TechPro Desktop": {
        "name": "TechPro Desktop",
        "category": "Computers and Laptops",
        "brand": "TechPro",
        "model_number": "TP-DT500",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["Intel Core i7 processor", "16GB RAM", "1TB HDD", "NVIDIA GeForce GTX 1660"],
        "description": "A powerful desktop computer for work and play.",
        "price": 999.99
    },
    "BlueWave Chromebook": {
        "name": "BlueWave Chromebook",
        "category": "Computers and Laptops",
        "brand": "BlueWave",
        "model_number": "BW-CB100",
        "warranty": "1 year",
        "rating": 4.1,
        "features": ["11.6-inch display", "4GB RAM", "32GB eMMC", "Chrome OS"],
        "description": "A compact and affordable Chromebook for everyday tasks.",
        "price": 249.99
    },
    "SmartX ProPhone": {
        "name": "SmartX ProPhone",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-PP10",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["6.1-inch display", "128GB storage", "12MP dual camera", "5G"],
        "description": "A powerful smartphone with advanced camera features.",
        "price": 899.99
    },
    "MobiTech PowerCase": {
        "name": "MobiTech PowerCase",
        "category": "Smartphones and Accessories",
        "brand": "MobiTech",
        "model_number": "MT-PC20",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["5000mAh battery", "Wireless charging", "Compatible with SmartX ProPhone"],
        "description": "A protective case with built-in battery for extended usage.",
        "price": 59.99
    },
    "SmartX MiniPhone": {
        "name": "SmartX MiniPhone",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-MP5",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["4.7-inch display", "64GB storage", "8MP camera", "4G"],
        "description": "A compact and affordable smartphone for basic tasks.",
        "price": 399.99
    },
    "MobiTech Wireless Charger": {
        "name": "MobiTech Wireless Charger",
        "category": "Smartphones and Accessories",
        "brand": "MobiTech",
        "model_number": "MT-WC10",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["10W fast charging", "Qi-compatible", "LED indicator", "Compact design"],
        "description": "A convenient wireless charger for a clutter-free workspace.",
        "price": 29.99
    },
    "SmartX EarBuds": {
        "name": "SmartX EarBuds",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-EB20",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["True wireless", "Bluetooth 5.0", "Touch controls", "24-hour battery life"],
        "description": "Experience true wireless freedom with these comfortable earbuds.",
        "price": 99.99
    },

    "CineView 4K TV": {
        "name": "CineView 4K TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-4K55",
        "warranty": "2 years",
        "rating": 4.8,
        "features": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "description": "A stunning 4K TV with vibrant colors and smart features.",
        "price": 599.99
    },
    "SoundMax Home Theater": {
        "name": "SoundMax Home Theater",
        "category": "Televisions and Home Theater Systems",
        "brand": "SoundMax",
        "model_number": "SM-HT100",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth"],
        "description": "A powerful home theater system for an immersive audio experience.",
        "price": 399.99
    },
    "CineView 8K TV": {
        "name": "CineView 8K TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-8K65",
        "warranty": "2 years",
        "rating": 4.9,
        "features": ["65-inch display", "8K resolution", "HDR", "Smart TV"],
        "description": "Experience the future of television with this stunning 8K TV.",
        "price": 2999.99
    },
    "SoundMax Soundbar": {
        "name": "SoundMax Soundbar",
        "category": "Televisions and Home Theater Systems",
        "brand": "SoundMax",
        "model_number": "SM-SB50",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth"],
        "description": "Upgrade your TV's audio with this sleek and powerful soundbar.",
        "price": 199.99
    },
    "CineView OLED TV": {
        "name": "CineView OLED TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-OLED55",
        "warranty": "2 years",
        "rating": 4.7,
        "features": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "description": "Experience true blacks and vibrant colors with this OLED TV.",
        "price": 1499.99
    },

    "GameSphere X": {
        "name": "GameSphere X",
        "category": "Gaming Consoles and Accessories",
        "brand": "GameSphere",
        "model_number": "GS-X",
        "warranty": "1 year",
        "rating": 4.9,
        "features": ["4K gaming", "1TB storage", "Backward compatibility", "Online multiplayer"],
        "description": "A next-generation gaming console for the ultimate gaming experience.",
        "price": 499.99
    },
    "ProGamer Controller": {
        "name": "ProGamer Controller",
        "category": "Gaming Consoles and Accessories",
        "brand": "ProGamer",
        "model_number": "PG-C100",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["Ergonomic design", "Customizable buttons", "Wireless", "Rechargeable battery"],
        "description": "A high-quality gaming controller for precision and comfort.",
        "price": 59.99
    },
    "GameSphere Y": {
        "name": "GameSphere Y",
        "category": "Gaming Consoles and Accessories",
        "brand": "GameSphere",
        "model_number": "GS-Y",
        "warranty": "1 year",
        "rating": 4.8,
        "features": ["4K gaming", "500GB storage", "Backward compatibility", "Online multiplayer"],
        "description": "A compact gaming console with powerful performance.",
        "price": 399.99
    },
    "ProGamer Racing Wheel": {
        "name": "ProGamer Racing Wheel",
        "category": "Gaming Consoles and Accessories",
        "brand": "ProGamer",
        "model_number": "PG-RW200",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["Force feedback", "Adjustable pedals", "Paddle shifters", "Compatible with GameSphere X"],
        "description": "Enhance your racing games with this realistic racing wheel.",
        "price": 249.99
    },
    "GameSphere VR Headset": {
        "name": "GameSphere VR Headset",
        "category": "Gaming Consoles and Accessories",
        "brand": "GameSphere",
        "model_number": "GS-VR",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["Immersive VR experience", "Built-in headphones", "Adjustable headband", "Compatible with GameSphere X"],
        "description": "Step into the world of virtual reality with this comfortable VR headset.",
        "price": 299.99
    },

    "AudioPhonic Noise-Canceling Headphones": {
        "name": "AudioPhonic Noise-Canceling Headphones",
        "category": "Audio Equipment",
        "brand": "AudioPhonic",
        "model_number": "AP-NC100",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["Active noise-canceling", "Bluetooth", "20-hour battery life", "Comfortable fit"],
        "description": "Experience immersive sound with these noise-canceling headphones.",
        "price": 199.99
    },
    "WaveSound Bluetooth Speaker": {
        "name": "WaveSound Bluetooth Speaker",
        "category": "Audio Equipment",
        "brand": "WaveSound",
        "model_number": "WS-BS50",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["Portable", "10-hour battery life", "Water-resistant", "Built-in microphone"],
        "description": "A compact and versatile Bluetooth speaker for music on the go.",
        "price": 49.99
    },
    "AudioPhonic True Wireless Earbuds": {
        "name": "AudioPhonic True Wireless Earbuds",
        "category": "Audio Equipment",
        "brand": "AudioPhonic",
        "model_number": "AP-TW20",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["True wireless", "Bluetooth 5.0", "Touch controls", "18-hour battery life"],
        "description": "Enjoy music without wires with these comfortable true wireless earbuds.",
        "price": 79.99
    },
    "WaveSound Soundbar": {
        "name": "WaveSound Soundbar",
        "category": "Audio Equipment",
        "brand": "WaveSound",
        "model_number": "WS-SB40",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["2.0 channel", "80W output", "Bluetooth", "Wall-mountable"],
        "description": "Upgrade your TV's audio with this slim and powerful soundbar.",
        "price": 99.99
    },
    "AudioPhonic Turntable": {
        "name": "AudioPhonic Turntable",
        "category": "Audio Equipment",
        "brand": "AudioPhonic",
        "model_number": "AP-TT10",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["3-speed", "Built-in speakers", "Bluetooth", "USB recording"],
        "description": "Rediscover your vinyl collection with this modern turntable.",
        "price": 149.99
    },

    "FotoSnap DSLR Camera": {
        "name": "FotoSnap DSLR Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-DSLR200",
        "warranty": "1 year",
        "rating": 4.7,
        "features": ["24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses"],
        "description": "Capture stunning photos and videos with this versatile DSLR camera.",
        "price": 599.99
    },
    "ActionCam 4K": {
        "name": "ActionCam 4K",
        "category": "Cameras and Camcorders",
        "brand": "ActionCam",
        "model_number": "AC-4K",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["4K video", "Waterproof", "Image stabilization", "Wi-Fi"],
        "description": "Record your adventures with this rugged and compact 4K action camera.",
        "price": 299.99
    },
    "FotoSnap Mirrorless Camera": {
        "name": "FotoSnap Mirrorless Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-ML100",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["20.1MP sensor", "4K video", "3-inch touchscreen", "Interchangeable lenses"],
        "description": "A compact and lightweight mirrorless camera with advanced features.",
        "price": 799.99
    },
    "ZoomMaster Camcorder": {
        "name": "ZoomMaster Camcorder",
        "category": "Cameras and Camcorders",
        "brand": "ZoomMaster",
        "model_number": "ZM-CM50",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["1080p video", "30x optical zoom", "3-inch LCD", "Image stabilization"],
        "description": "Capture life's moments with this easy-to-use camcorder.",
        "price": 249.99
    },
    "FotoSnap Instant Camera": {
        "name": "FotoSnap Instant Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-IC10",
        "warranty": "1 year",
        "rating": 4.1,
        "features": ["Instant prints", "Built-in flash", "Selfie mirror", "Battery-powered"],
        "description": "Create instant memories with this fun and portable instant camera.",
        "price": 69.99
    }
}

In [ ]:
def get_product_by_name(name):
    """
    製品名で製品を取得

    パラメータ:
    name: 製品名
    """
    return products.get(name, None)

def get_products_by_category(category):
    """
    カテゴリーで製品を取得

    パラメータ:
    category: 製品カテゴリー
    """
    return [product for product in products.values() if product["category"] == category]

In [7]:
print(get_product_by_name("TechPro Ultrabook"))

{'name': 'TechPro Ultrabook', 'category': 'Computers and Laptops', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 processor'], 'description': 'A sleek and lightweight ultrabook for everyday use.', 'price': 799.99}


In [8]:
print(get_products_by_category("Computers and Laptops"))

[{'name': 'TechPro Ultrabook', 'category': 'Computers and Laptops', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 processor'], 'description': 'A sleek and lightweight ultrabook for everyday use.', 'price': 799.99}, {'name': 'BlueWave Gaming Laptop', 'category': 'Computers and Laptops', 'brand': 'BlueWave', 'model_number': 'BW-GL200', 'warranty': '2 years', 'rating': 4.7, 'features': ['15.6-inch display', '16GB RAM', '512GB SSD', 'NVIDIA GeForce RTX 3060'], 'description': 'A high-performance gaming laptop for an immersive experience.', 'price': 1199.99}, {'name': 'PowerLite Convertible', 'category': 'Computers and Laptops', 'brand': 'PowerLite', 'model_number': 'PL-CV300', 'warranty': '1 year', 'rating': 4.3, 'features': ['14-inch touchscreen', '8GB RAM', '256GB SSD', '360-degree hinge'], 'description': 'A versatile convertible laptop with a responsive touchscreen.', 'price': 

In [9]:
print(user_message_1)


 tell me about the smartx pro phone and  the fotosnap camera, the dslr one.  Also tell me about your tvs 


In [10]:
print(category_and_product_response_1)

[
    {'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']},
    {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']},
    {'category': 'Televisions and Home Theater Systems'}
]


In [ ]:
# 製品情報
products = {
    "TechPro Ultrabook": {
        "name": "TechPro ウルトラブック",
        "category": "コンピューターとノートパソコン",
        "brand": "TechPro",
        "model_number": "TP-UB100",
        "warranty": "1年",
        "rating": 4.5,
        "features": ["13.3インチディスプレイ", "8GB RAM", "256GB SSD", "Intel Core i5プロセッサー"],
        "description": "日常使いに適したスタイリッシュで軽量なウルトラブック。",
        "price": 799.99
    },
    "BlueWave Gaming Laptop": {
        "name": "BlueWave ゲーミングノートパソコン",
        "category": "コンピューターとノートパソコン",
        "brand": "BlueWave",
        "model_number": "BW-GL200",
        "warranty": "2年",
        "rating": 4.7,
        "features": ["15.6インチディスプレイ", "16GB RAM", "512GB SSD", "NVIDIA GeForce RTX 3060"],
        "description": "没入感のある体験を提供する高性能ゲーミングノートパソコン。",
        "price": 1199.99
    },
    "PowerLite Convertible": {
        "name": "PowerLite コンバーチブル",
        "category": "コンピューターとノートパソコン",
        "brand": "PowerLite",
        "model_number": "PL-CV300",
        "warranty": "1年",
        "rating": 4.3,
        "features": ["14インチタッチスクリーン", "8GB RAM", "256GB SSD", "360度ヒンジ"],
        "description": "レスポンシブなタッチスクリーンを備えた多機能コンバーチブルノートパソコン。",
        "price": 699.99
    },
    "TechPro Desktop": {
        "name": "TechPro デスクトップ",
        "category": "コンピューターとノートパソコン",
        "brand": "TechPro",
        "model_number": "TP-DT500",
        "warranty": "1年",
        "rating": 4.4,
        "features": ["Intel Core i7プロセッサー", "16GB RAM", "1TB HDD", "NVIDIA GeForce GTX 1660"],
        "description": "仕事と遊びに適した強力なデスクトップコンピューター。",
        "price": 999.99
    },
    "BlueWave Chromebook": {
        "name": "BlueWave Chromebook",
        "category": "コンピューターとノートパソコン",
        "brand": "BlueWave",
        "model_number": "BW-CB100",
        "warranty": "1年",
        "rating": 4.1,
        "features": ["11.6インチディスプレイ", "4GB RAM", "32GB eMMC", "Chrome OS"],
        "description": "日常的なタスクに適したコンパクトで手頃な価格のChromebook。",
        "price": 249.99
    },
    "SmartX ProPhone": {
        "name": "SmartX ProPhone",
        "category": "スマートフォンとアクセサリー",
        "brand": "SmartX",
        "model_number": "SX-PP10",
        "warranty": "1年",
        "rating": 4.6,
        "features": ["6.1インチディスプレイ", "128GBストレージ", "12MPデュアルカメラ", "5G"],
        "description": "高度なカメラ機能を備えた強力なスマートフォン。",
        "price": 899.99
    },
    "MobiTech PowerCase": {
        "name": "MobiTech PowerCase",
        "category": "スマートフォンとアクセサリー",
        "brand": "MobiTech",
        "model_number": "MT-PC20",
        "warranty": "1年",
        "rating": 4.3,
        "features": ["5000mAhバッテリー", "ワイヤレス充電", "SmartX ProPhone対応"],
        "description": "使用時間を延長する内蔵バッテリー付き保護ケース。",
        "price": 59.99
    },
    "SmartX MiniPhone": {
        "name": "SmartX MiniPhone",
        "category": "スマートフォンとアクセサリー",
        "brand": "SmartX",
        "model_number": "SX-MP5",
        "warranty": "1年",
        "rating": 4.2,
        "features": ["4.7インチディスプレイ", "64GBストレージ", "8MPカメラ", "4G"],
        "description": "基本的なタスクに適したコンパクトで手頃な価格のスマートフォン。",
        "price": 399.99
    },
    "MobiTech Wireless Charger": {
        "name": "MobiTech ワイヤレス充電器",
        "category": "スマートフォンとアクセサリー",
        "brand": "MobiTech",
        "model_number": "MT-WC10",
        "warranty": "1年",
        "rating": 4.5,
        "features": ["10W高速充電", "Qi対応", "LEDインジケーター", "コンパクトデザイン"],
        "description": "作業スペースを整理整頓できる便利なワイヤレス充電器。",
        "price": 29.99
    },
    "SmartX EarBuds": {
        "name": "SmartX イヤホン",
        "category": "スマートフォンとアクセサリー",
        "brand": "SmartX",
        "model_number": "SX-EB20",
        "warranty": "1年",
        "rating": 4.4,
        "features": ["完全ワイヤレス", "Bluetooth 5.0", "タッチコントロール", "24時間バッテリー"],
        "description": "これらの快適なイヤホンで真のワイヤレス自由を体験してください。",
        "price": 99.99
    },

    "CineView 4K TV": {
        "name": "CineView 4K テレビ",
        "category": "テレビとホームシアターシステム",
        "brand": "CineView",
        "model_number": "CV-4K55",
        "warranty": "2年",
        "rating": 4.8,
        "features": ["55インチディスプレイ", "4K解像度", "HDR", "スマートTV"],
        "description": "鮮やかな色彩とスマート機能を備えた素晴らしい4Kテレビ。",
        "price": 599.99
    },
    "SoundMax Home Theater": {
        "name": "SoundMax ホームシアター",
        "category": "テレビとホームシアターシステム",
        "brand": "SoundMax",
        "model_number": "SM-HT100",
        "warranty": "1年",
        "rating": 4.4,
        "features": ["5.1チャンネル", "1000W出力", "ワイヤレスサブウーファー", "Bluetooth"],
        "description": "没入感のあるオーディオ体験を提供する強力なホームシアターシステム。",
        "price": 399.99
    },
    "CineView 8K TV": {
        "name": "CineView 8K テレビ",
        "category": "テレビとホームシアターシステム",
        "brand": "CineView",
        "model_number": "CV-8K65",
        "warranty": "2年",
        "rating": 4.9,
        "features": ["65インチディスプレイ", "8K解像度", "HDR", "スマートTV"],
        "description": "この素晴らしい8Kテレビで未来を体験してください。",
        "price": 2999.99
    },
    "SoundMax Soundbar": {
        "name": "SoundMax サウンドバー",
        "category": "テレビとホームシアターシステム",
        "brand": "SoundMax",
        "model_number": "SM-SB50",
        "warranty": "1年",
        "rating": 4.3,
        "features": ["2.1チャンネル", "300W出力", "ワイヤレスサブウーファー", "Bluetooth"],
        "description": "このスリムで強力なサウンドバーでテレビのオーディオをアップグレードしてください。",
        "price": 199.99
    },
    "CineView OLED TV": {
        "name": "CineView OLED テレビ",
        "category": "テレビとホームシアターシステム",
        "brand": "CineView",
        "model_number": "CV-OLED55",
        "warranty": "2年",
        "rating": 4.7,
        "features": ["55インチディスプレイ", "4K解像度", "HDR", "スマートTV"],
        "description": "このOLEDテレビで真の黒と鮮やかな色彩を体験してください。",
        "price": 1499.99
    },

    "GameSphere X": {
        "name": "GameSphere X",
        "category": "ゲーム機とアクセサリー",
        "brand": "GameSphere",
        "model_number": "GS-X",
        "warranty": "1年",
        "rating": 4.9,
        "features": ["4Kゲーミング", "1TBストレージ", "下位互換性", "オンラインマルチプレイヤー"],
        "description": "究極のゲーム体験を提供する次世代ゲーム機。",
        "price": 499.99
    },
    "ProGamer Controller": {
        "name": "ProGamer コントローラー",
        "category": "ゲーム機とアクセサリー",
        "brand": "ProGamer",
        "model_number": "PG-C100",
        "warranty": "1年",
        "rating": 4.2,
        "features": ["エルゴノミックデザイン", "カスタマイズ可能なボタン", "ワイヤレス", "充電式バッテリー"],
        "description": "精密で快適な高品質ゲーミングコントローラー。",
        "price": 59.99
    },
    "GameSphere Y": {
        "name": "GameSphere Y",
        "category": "ゲーム機とアクセサリー",
        "brand": "GameSphere",
        "model_number": "GS-Y",
        "warranty": "1年",
        "rating": 4.8,
        "features": ["4Kゲーミング", "500GBストレージ", "下位互換性", "オンラインマルチプレイヤー"],
        "description": "強力なパフォーマンスを持つコンパクトなゲーム機。",
        "price": 399.99
    },
    "ProGamer Racing Wheel": {
        "name": "ProGamer レーシングホイール",
        "category": "ゲーム機とアクセサリー",
        "brand": "ProGamer",
        "model_number": "PG-RW200",
        "warranty": "1年",
        "rating": 4.5,
        "features": ["フォースフィードバック", "調整可能なペダル", "パドルシフター", "GameSphere X対応"],
        "description": "このリアルなレーシングホイールでレーシングゲームを強化してください。",
        "price": 249.99
    },
    "GameSphere VR Headset": {
        "name": "GameSphere VRヘッドセット",
        "category": "ゲーム機とアクセサリー",
        "brand": "GameSphere",
        "model_number": "GS-VR",
        "warranty": "1年",
        "rating": 4.6,
        "features": ["没入型VR体験", "内蔵ヘッドフォン", "調整可能なヘッドバンド", "GameSphere X対応"],
        "description": "この快適なVRヘッドセットでバーチャルリアリティの世界に足を踏み入れてください。",
        "price": 299.99
    },

    "AudioPhonic Noise-Canceling Headphones": {
        "name": "AudioPhonic ノイズキャンセリングヘッドフォン",
        "category": "オーディオ機器",
        "brand": "AudioPhonic",
        "model_number": "AP-NC100",
        "warranty": "1年",
        "rating": 4.6,
        "features": ["アクティブノイズキャンセリング", "Bluetooth", "20時間バッテリー", "快適なフィット"],
        "description": "これらのノイズキャンセリングヘッドフォンで没入感のあるサウンドを体験してください。",
        "price": 199.99
    },
    "WaveSound Bluetooth Speaker": {
        "name": "WaveSound Bluetoothスピーカー",
        "category": "オーディオ機器",
        "brand": "WaveSound",
        "model_number": "WS-BS50",
        "warranty": "1年",
        "rating": 4.5,
        "features": ["ポータブル", "10時間バッテリー", "防水", "内蔵マイク"],
        "description": "外出先で音楽を楽しめるコンパクトで多用途なBluetoothスピーカー。",
        "price": 49.99
    },
    "AudioPhonic True Wireless Earbuds": {
        "name": "AudioPhonic 完全ワイヤレスイヤホン",
        "category": "オーディオ機器",
        "brand": "AudioPhonic",
        "model_number": "AP-TW20",
        "warranty": "1年",
        "rating": 4.4,
        "features": ["完全ワイヤレス", "Bluetooth 5.0", "タッチコントロール", "18時間バッテリー"],
        "description": "これらの快適な完全ワイヤレスイヤホンでケーブルなしで音楽を楽しんでください。",
        "price": 79.99
    },
    "WaveSound Soundbar": {
        "name": "WaveSound サウンドバー",
        "category": "オーディオ機器",
        "brand": "WaveSound",
        "model_number": "WS-SB40",
        "warranty": "1年",
        "rating": 4.3,
        "features": ["2.0チャンネル", "80W出力", "Bluetooth", "壁掛け可能"],
        "description": "このスリムで強力なサウンドバーでテレビのオーディオをアップグレードしてください。",
        "price": 99.99
    },
    "AudioPhonic Turntable": {
        "name": "AudioPhonic ターンテーブル",
        "category": "オーディオ機器",
        "brand": "AudioPhonic",
        "model_number": "AP-TT10",
        "warranty": "1年",
        "rating": 4.2,
        "features": ["3速", "内蔵スピーカー", "Bluetooth", "USB録音"],
        "description": "このモダンなターンテーブルでビニールコレクションを再発見してください。",
        "price": 149.99
    },

    "FotoSnap DSLR Camera": {
        "name": "FotoSnap デジタル一眼レフカメラ",
        "category": "カメラとビデオカメラ",
        "brand": "FotoSnap",
        "model_number": "FS-DSLR200",
        "warranty": "1年",
        "rating": 4.7,
        "features": ["24.2MPセンサー", "1080pビデオ", "3インチLCD", "交換可能レンズ"],
        "description": "この多用途デジタル一眼レフカメラで素晴らしい写真とビデオを撮影してください。",
        "price": 599.99
    },
    "ActionCam 4K": {
        "name": "ActionCam 4K",
        "category": "カメラとビデオカメラ",
        "brand": "ActionCam",
        "model_number": "AC-4K",
        "warranty": "1年",
        "rating": 4.4,
        "features": ["4Kビデオ", "防水", "画像安定化", "Wi-Fi"],
        "description": "この頑丈でコンパクトな4Kアクションカメラで冒険を記録してください。",
        "price": 299.99
    },
    "FotoSnap Mirrorless Camera": {
        "name": "FotoSnap ミラーレスカメラ",
        "category": "カメラとビデオカメラ",
        "brand": "FotoSnap",
        "model_number": "FS-ML100",
        "warranty": "1年",
        "rating": 4.6,
        "features": ["20.1MPセンサー", "4Kビデオ", "3インチタッチスクリーン", "交換可能レンズ"],
        "description": "高度な機能を備えたコンパクトで軽量なミラーレスカメラ。",
        "price": 799.99
    },
    "ZoomMaster Camcorder": {
        "name": "ZoomMaster ビデオカメラ",
        "category": "カメラとビデオカメラ",
        "brand": "ZoomMaster",
        "model_number": "ZM-CM50",
        "warranty": "1年",
        "rating": 4.3,
        "features": ["1080pビデオ", "30倍光学ズーム", "3インチLCD", "画像安定化"],
        "description": "この使いやすいビデオカメラで人生の瞬間を捉えてください。",
        "price": 249.99
    },
    "FotoSnap Instant Camera": {
        "name": "FotoSnap インスタントカメラ",
        "category": "カメラとビデオカメラ",
        "brand": "FotoSnap",
        "model_number": "FS-IC10",
        "warranty": "1年",
        "rating": 4.1,
        "features": ["インスタントプリント", "内蔵フラッシュ", "セルフィーミラー", "バッテリー駆動"],
        "description": "この楽しくてポータブルなインスタントカメラで瞬間の思い出を作ってください。",
        "price": 69.99
    }
}

In [17]:
print(get_product_by_name("TechPro Ultrabook"))

{'name': 'TechPro 超极本', 'category': '电脑和笔记本', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 处理器'], 'description': '一款时尚轻便的超极本，适合日常使用。', 'price': 799.99}


In [ ]:
print(get_products_by_category("コンピューターとノートパソコン"))

In [ ]:
print(user_message_1)

In [21]:
print(category_and_product_response_1)

[{'category': '智能手机和配件', 'products': ['SmartX ProPhone']}, {'category': '相机和摄像机', 'products': ['FotoSnap DSLR Camera', 'FotoSnap Mirrorless Camera']}, {'category': '电视和家庭影院系统', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]


### 2.3 Python文字列をPython辞書リストとして読み込む

In [ ]:
import json 

def read_string_to_list(input_string):
    """
    入力文字列をPythonリストに変換します。

    パラメータ:
    input_string: 入力文字列、有効なJSON形式である必要があります。

    戻り値:
    list または None: 入力文字列が有効な場合は対応するPythonリストを返し、そうでない場合はNoneを返します。
    """
    if input_string is None:
        return None

    try:
        # JSON形式の要件を満たすために、入力文字列のシングルクォートをダブルクォートに置き換えます
        input_string = input_string.replace("'", "\"")  
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("エラー: 無効なJSON文字列")
        return None

In [15]:
category_and_product_list = read_string_to_list(category_and_product_response_1)
print(category_and_product_list)

[{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems'}]


#### 2.3.1 関連製品とカテゴリーの詳細情報の呼び出し

In [ ]:
def generate_output_string(data_list):
    """
    入力データリストに基づいて製品またはカテゴリー情報を含む文字列を生成します。

    パラメータ:
    data_list: 辞書を含むリスト、各辞書には"products"または"category"のキーが含まれている必要があります。

    戻り値:
    output_string: 製品またはカテゴリー情報を含む文字列。
    """
    output_string = ""
    
    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4, ensure_ascii=False) + "\n"
                    else:
                        print(f"エラー: 製品 '{product_name}' が見つかりません")
            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4, ensure_ascii=False) + "\n"
            else:
                print("エラー: 無効なオブジェクト形式")
        except Exception as e:
            print(f"エラー: {e}")

    return output_string

In [16]:
product_information_for_user_message_1 = generate_output_string(category_and_product_list)
print(product_information_for_user_message_1)

{
    "name": "SmartX ProPhone",
    "category": "Smartphones and Accessories",
    "brand": "SmartX",
    "model_number": "SX-PP10",
    "warranty": "1 year",
    "rating": 4.6,
    "features": [
        "6.1-inch display",
        "128GB storage",
        "12MP dual camera",
        "5G"
    ],
    "description": "A powerful smartphone with advanced camera features.",
    "price": 899.99
}
{
    "name": "FotoSnap DSLR Camera",
    "category": "Cameras and Camcorders",
    "brand": "FotoSnap",
    "model_number": "FS-DSLR200",
    "warranty": "1 year",
    "rating": 4.7,
    "features": [
        "24.2MP sensor",
        "1080p video",
        "3-inch LCD",
        "Interchangeable lenses"
    ],
    "description": "Capture stunning photos and videos with this versatile DSLR camera.",
    "price": 599.99
}
{
    "name": "CineView 4K TV",
    "category": "Televisions and Home Theater Systems",
    "brand": "CineView",
    "model_number": "CV-4K55",
    "warranty": "2 years",
    "ratin

### 2.4 詳細な製品情報に基づくユーザークエリへの回答生成

In [17]:
system_message = f"""
You are a customer service assistant for a \
large electronic store. \
Respond in a friendly and helpful tone, \
with very concise answers. \
Make sure to ask the user relevant follow up questions.
"""
user_message_1 = f"""
tell me about the smartx pro phone and \
the fotosnap camera, the dslr one. \
Also tell me about your tvs"""
messages =  [  
{'role':'system',
 'content': system_message},   
{'role':'user',
 'content': user_message_1},  
{'role':'assistant',
 'content': f"""Relevant product information:\n\
 {product_information_for_user_message_1}"""},   
]
final_response = get_completion_from_messages(messages)
print(final_response)

The SmartX ProPhone has a 6.1-inch display, 128GB storage, 12MP dual camera, and 5G. The FotoSnap DSLR Camera has a 24.2MP sensor, 1080p video, 3-inch LCD, and interchangeable lenses. We have a variety of TVs, including the CineView 4K TV with a 55-inch display, 4K resolution, HDR, and smart TV features. We also have the SoundMax Home Theater system with 5.1 channel, 1000W output, wireless subwoofer, and Bluetooth. Do you have any specific questions about these products or any other products we offer?


In [ ]:
system_message = f"""
あなたは大型電器店のカスタマーサービスアシスタントです。
フレンドリーで親切なトーンで応答してください。
とても簡潔な回答を心がけてください。
ユーザーに関連するフォローアップの質問をしてください。
"""
user_message_1 = f"""
SmartX ProPhoneスマートフォンとFotoSnapカメラ、一眼レフカメラについて教えてください。
また、テレビ製品についての情報も教えてください。"""
messages =  [  
{'role':'system',
 'content': system_message},   
{'role':'user',
 'content': user_message_1},  
{'role':'assistant',
 'content': f"""関連製品情報:\n\
 {product_information_for_user_message_1}"""},   
]
final_response = get_completion_from_messages(messages)
print(final_response)

一連のステップを通じてユーザークエリに関連する情報をロードし、質問に効果的に答えるために必要なコンテキストをモデルに提供する方法について説明しました。

なぜプロンプトに製品説明を選択的にロードし、すべての製品説明を含めてモデルに必要な情報を使用させないのか、疑問に思われるかもしれません。

これにはいくつかの理由があります。

まず、過多の製品説明を含めると、モデルがコンテキストを処理する際に混乱する可能性があります。これは、一度に大量の情報を処理しようとする人と同じです。もちろん、GPT-4のような高度なモデルにとっては、この理由はそれほど重要ではありません。特に、この例のようにコンテキストが良い構造を持っている場合、モデルは明らかに関連しない情報を巧みに無視するのに十分賢いです。

次の理由はより説得力があります。

まず、すべての製品説明を含めると、モデルがコンテキストについてより混乱する可能性があります。これは、一度に大量の情報を処理しようとする人と同じです。もちろん、GPT-4のような高度なモデルにとっては、この問題はそれほど関連性がありません。特に、この例のようにコンテキストが良い構造を持っている場合、モデルは明らかに関連しない情報を無視するのに十分賢いです。次の理由はより説得力があります。

2番目の理由は、言語モデルにはコンテキストの制限、つまり入力と出力として許可される固定数のトークンがあることです。巨大な製品カタログがある場合、すべての説明をコンテキストウィンドウに収めることさえできません。

最後の理由は、すべての製品説明を含めると、モデルが過学習する可能性があることです。モデルはクエリに関連する情報だけでなく、すべての製品説明を記憶してしまう可能性があります。これにより、新しいクエリを処理する際にモデルのパフォーマンスが低下する可能性があります。

言語モデルを使用する場合、トークンごとに料金を支払うため、コストがかかる可能性があります。したがって、情報を選択的にロードすることで、応答を生成するコストを削減できます。一般的に、モデルのコンテキストに情報を動的にロードするタイミングを判断し、モデルがいつより多くの情報を必要とするかを決定できるようにすることは、これらのモデルの能力を強化する最良の方法の1つです。

再度強調しますが、言語モデルを有用な結論を導き出し、有用なタスクを実行するために必要なコンテキストを必要とする推論エージェントと見なすべきです。したがって、この場合、モデルに製品情報を提供し、その製品情報に基づいて推論を行い、ユーザーのために有用な答えを作成できるようにする必要があります。

この例では、製品名で製品説明を取得したり、カテゴリー名でカテゴリー製品を取得したりする特定の関数または関数の呼び出しのみを追加しました。しかし、モデルは実際にさまざまな異なるツールをいつ使用するかを決定し、それらを正しく使用するのが得意です。これがChatGPTプラグインの背後にある考え方です。モデルがアクセスできるツールとその動作を伝え、特定のソースから情報を取得したり、他の適切なアクションを実行したりする必要があるときに、モデルがそれらを使用することを選択します。この例では、正確な製品名とカテゴリー名の一致によってのみ情報を検索できますが、より高度な情報検索技術があります。情報を検索する最も効果的な方法の1つは、固有表現認識や関係抽出などの自然言語処理技術を使用することです。

もう1つの方法は、テキスト埋め込み（Embedding）を使用して情報を取得することです。埋め込みは、大規模なコーパスに対する効率的な知識検索を実装し、特定のクエリに関連する情報を見つけるために使用できます。テキスト埋め込みを使用する主な利点は、あいまい検索または意味検索を実現できることで、正確なキーワードを使用せずに関連情報を見つけることができます。したがって、この例では、製品の正確な名前は必ずしも必要ではなく、**「携帯電話」**のようなより一般的なクエリで検索できます。さまざまなアプリケーションで埋め込みを使用する方法を紹介する包括的なコースを近日中に開始する予定ですので、ご期待ください。

次の章では、言語モデルの出力を評価する方法について説明します。